# OAuth Token Authentication as a user

## Overview

Details related to OAuth Token Authentication are described: <a href="https://learn.microsoft.com/en-us/azure/databricks/dev-tools/auth/oauth-u2m" target="_blank">here</a>.  

### Step 1: Generate an OAuth code verifier and code challenge pair
To manually generate and use access tokens for OAuth U2M authentication, you must first have:
- a) an OAuth code verifier. Used in step 3 with (c) to generate the OAuth access token
- b) an OAuth code challenge, derived from the code verifier. Used in Step 2 to generate an OAuth authorization code (c).

In [0]:
import uuid, hashlib, base64

# Generate a UUID.
uuid1 = uuid.uuid4()

# Convert the UUID to a string.
uuid_str1 = str(uuid1).upper()

# Create the code verifier.
code_verifier = uuid_str1 + "-" + uuid_str1

# Create the code challenge based on the code verifier.
code_challenge = base64.urlsafe_b64encode(hashlib.sha256(code_verifier.encode()).digest()).decode('utf-8')

# Remove all padding from the code challenge.
code_challenge = code_challenge.replace('=', '')

# Print the code verifier and the code challenge.
# Use these in your calls to manually generate
# access tokens for OAuth U2M authentication.
print(f"code_verifier:  {code_verifier}")
print(f"code_challenge: {code_challenge}")

### Step 2: Generate an authorization code
You use an OAuth authorization code to generate an Azure Databricks OAuth access token. 
The authorization code expires immediately after you use it to generate an Azure Databricks OAuth access token. 
The scope of the authorization code depends on the level that you generate it from. 
You can generate an authorization code at either the Azure Databricks account level or workspace level, as follows:
- generate an authorization code at the account level \
to call account-level and workspace-level REST APIs within accounts and workspaces that your Azure Databricks user account has access to

- generate an authorization code at the workspace level for only that workspace \
to call REST APIs within only one workspace that your user account has access to 



#### Generate an account-level authorization code
<pre>
https://accounts.azuredatabricks.net/oidc/accounts/<account-id>/v1/authorize
?client_id=databricks-cli
&redirect_uri=<redirect-url>
&response_type=code
&state=<state>
&code_challenge=<code-challenge>
&code_challenge_method=S256
&scope=all-apis+offline_access
</pre>

In [0]:
auth_code = <>

#### Generate a workspace-level authorization code
<pre>
https://<databricks-instance>/oidc/v1/authorize
?client_id=databricks-cli
&redirect_uri=<redirect-url>
&response_type=code
&state=<state>
&code_challenge=<code-challenge>
&code_challenge_method=S256
&scope=all-apis+offline_access
</pre>

### Step 3: Use the authorization code to generate an OAuth access token
You use the OAuth authorization code from the previous step to generate an Azure Databricks OAuth access token, as follows:

- use the account-level authorization code to generate an access token at the account level: \
to call account-level and workspace-level REST APIs within accounts and workspaces that your Azure Databricks user account has access to

- use the workspace-level authorization code to generate an access token at the workspace level for only that workspace: \
To call REST APIs within only one workspace that your user account has access to

#### Generate an account-level access token
<pre>
curl --request POST \
https://accounts.azuredatabricks.net/oidc/accounts/<account-id>/v1/token \
--data "client_id=databricks-cli" \
--data "grant_type=authorization_code" \
--data "scope=all-apis offline_access" \
--data "redirect_uri=<redirect-url>" \
--data "code_verifier=<code-verifier>" \
--data "code=<authorization-code>"
</pre>

#### Generate a workspace-level access token
<pre>
curl --request POST \
https://<databricks-instance>/oidc/v1/token \
--data "client_id=databricks-cli" \
--data "grant_type=authorization_code" \
--data "scope=all-apis offline_access" \
--data "redirect_uri=<redirect-url>" \
--data "code_verifier=<code-verifier>" \
--data "code=<authorization-code>"
</pre>

### Step 4: Call a Databricks REST API
You use the account-level or workspace-level OAuth access token to authenticate to Azure Databricks account-level REST APIs and workspace-level REST APIs, depending on the access token’s scope. \
Your Azure Databricks user account must be an account admin to call account-level REST APIs.

#### Example account-level REST API request
<pre>
export OAUTH_TOKEN=<oauth-access-token>

curl --request GET --header "Authorization: Bearer $OAUTH_TOKEN" \
"https://accounts.azuredatabricks.net/api/2.0/accounts/<account-id>/workspaces"
</pre>

#### Example workspace-level REST API `request`
<pre>
export OAUTH_TOKEN=<oauth-access-token>

curl --request GET --header "Authorization: Bearer $OAUTH_TOKEN" \
"https://<databricks-instance>/api/2.0/clusters/list"
</pre>